# Performance Comparison with Lispminer

## LispMiner Setting:
Different values for support (BASE) and confidence (PIM) were tried.

![lispminerresults](./lisp.png)

## Package action-rules Setting
LispMiner generates the action rules in both ways for the target (Yes -> No, No -> Yes). To get the same behaviour in action-rules package, the mining is run twice. And post filtering is needed because of flexible attributes in LISp-Miner are strictly flexible.

In [1]:
import pandas as pd
from action_rules import ActionRules

stable_attributes = ["gender", "SeniorCitizen", "Partner"]
flexible_attributes = ["PhoneService",
                       "InternetService",
                       "OnlineSecurity",
                       "DeviceProtection",
                       "TechSupport",
                       "StreamingTV"]
target = 'Churn'
min_stable_attributes = 3
min_flexible_attributes = 2 
undesired_state = 'Yes'
desired_state = 'No'

pd.set_option('display.max_columns', None)
data_frame = pd.read_csv("./../data/telco.csv", sep=";")

def mining(support, confidence):
    rules = []
    # first run Yes -> No
    action_rules = ActionRules(
        min_stable_attributes=min_stable_attributes,
        min_flexible_attributes=min_flexible_attributes,
        min_undesired_support=support,
        min_undesired_confidence=confidence,
        min_desired_support=support,
        min_desired_confidence=confidence,
        verbose=False
    )
    action_rules.fit(
        data=data_frame,
        stable_attributes=stable_attributes,
        flexible_attributes=flexible_attributes,
        target=target,
        target_undesired_state=undesired_state,
        target_desired_state=desired_state,
        use_gpu=False,
        use_sparse_matrix=False,
    )
    rules = action_rules.get_rules().get_ar_notation()
    # second run No -> Yes
    action_rules = ActionRules(
        min_stable_attributes=min_stable_attributes,
        min_flexible_attributes=min_flexible_attributes,
        min_undesired_support=support,
        min_undesired_confidence=confidence,
        min_desired_support=support,
        min_desired_confidence=confidence,
        verbose=False
    )
    action_rules.fit(
        data=data_frame,
        stable_attributes=stable_attributes,
        flexible_attributes=flexible_attributes,
        target=target,
        target_undesired_state=desired_state,
        target_desired_state=undesired_state,
        use_gpu=False,
        use_sparse_matrix=False,
    )
    rules += action_rules.get_rules().get_ar_notation()
    rul = []
    for r in rules:
        if '*' not in r: # just strictly flexible
            rul.append(r)
    return rul

# Results

### Support 70, confidence 50%

LISp-Miner

![lispminerresults](./1.png)

action-rules

In [2]:
%timeit mining(70, 0.5)

372 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
len(mining(70, 0.5))

178

### Support 70, confidence 60%

LISp-Miner

![lispminerresults](./2.png)

action-rules

In [4]:
%timeit mining(70, 0.6)

341 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
len(mining(70, 0.6))

32

### Support 140, confidence 50%

LISp-Miner

![lispminerresults](./3.png)

action-rules

In [6]:
%timeit mining(140, 0.5)

208 ms ± 5.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
len(mining(140, 0.5))

70

### Support 140, confidence 60%

LISp-Miner

![lispminerresults](./4.png)

action-rules

In [8]:
%timeit mining(140, 0.6)

187 ms ± 225 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
len(mining(140, 0.6))

8

## Compare rules

LISp-Miner

action-rules

In [10]:
mining(140, 0.6)

['[(gender: Female) ∧ (SeniorCitizen: 0) ∧ (Partner: No) ∧ (InternetService: Fiber optic → No) ∧ (OnlineSecurity: No → No internet service) ∧ (DeviceProtection: No → No internet service)] ⇒ [Churn: Yes → No], support of undesired part: 193, confidence of undesired part: 0.6126984126984127, support of desired part: 301, confidence of desired part: 0.8985074626865671, uplift: 0.022863815241554546',
 '[(gender: Female) ∧ (SeniorCitizen: 0) ∧ (Partner: No) ∧ (InternetService: Fiber optic → No) ∧ (DeviceProtection: No → No internet service) ∧ (TechSupport: No → No internet service)] ⇒ [Churn: Yes → No], support of undesired part: 198, confidence of undesired part: 0.6130030959752322, support of desired part: 301, confidence of desired part: 0.8985074626865671, uplift: 0.023458456687173244',
 '[(gender: Female) ∧ (SeniorCitizen: 0) ∧ (Partner: No) ∧ (InternetService: Fiber optic → No) ∧ (OnlineSecurity: No → No internet service) ∧ (TechSupport: No → No internet service)] ⇒ [Churn: Yes → No],

# Results Table

In [13]:
data = {
    'Support, Confidence': ['70, 50%', '70, 60%', '140, 50%', '140, 60%'],
    'LISp-Miner Time (s)': [70, 24, 11, 7],
    'LISp-Miner Rules': [178, 32, 70, 8],
    'Action-Rules Time (ms)': [372, 341, 208, 187],
    'Action-Rules Rules': [178, 32, 70, 8]
}

# Create DataFrame
df = pd.DataFrame(data)

df['Action-Rules Speed (x)'] = (df['LISp-Miner Time (s)'] * 1000) / df['Action-Rules Time (ms)']
df

,"Support, Confidence",LISp-Miner Time (s),LISp-Miner Rules,Action-Rules Time (ms),Action-Rules Rules,Action-Rules Speed (x)
0,"70, 50%",70,178,372,178,188.172043
1,"70, 60%",24,32,341,32,70.381232
2,"140, 50%",11,70,208,70,52.884615
3,"140, 60%",7,8,187,8,37.433155
